In [ ]:
#####################################################################
#
#       aiplatform.CustomTrainingJob -> script_path
#
#####################################################################

In [ ]:
#####################################################################
#
# define the training script
#
#####################################################################

In [ ]:
%%writefile training_script.py

import argparse, os, pickle
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from google.cloud import storage

# parse args
parser = argparse.ArgumentParser()
parser.add_argument('--bucket_name', dest='bucket_name', default="", type=str, help='The GCS bucket to store model artifacts -> w/o gs://')
parser.add_argument('--train_ds', dest='train_ds', default="", type=str, help='file name of the training data')
args = parser.parse_args()

storage_client = storage.Client()

#########################################################################
# with CustomTrainingJob, FUSE is available, cant get write to work
#########################################################################
DATA_PATH = f"/gcs/{args.bucket_name}"
TRAIN_PATH = os.path.join(DATA_PATH, args.train_ds)

# load data from GCS using GCS fuse
df = pd.read_csv(TRAIN_PATH)
labels = df.pop("label").tolist()
data = df.values.tolist()

# fit the model
skmodel = GradientBoostingClassifier(n_estimators = 100, max_depth = 10, min_samples_split = 100, min_samples_leaf = 100)
skmodel.fit(data, labels)

# save model to GCS
MODEL_SAVE_PATH = os.environ['AIP_MODEL_DIR']
model_filename = "model.pkl"
with open(model_filename, 'wb') as model_file:
    pickle.dump(skmodel, model_file)

storage_path = os.path.join(MODEL_SAVE_PATH, model_filename)
blob = storage.blob.Blob.from_string(storage_path, client=storage_client)
blob.upload_from_filename(model_filename)

In [ ]:
#####################################################################
#
# kick off the custom training job
#
#####################################################################

In [ ]:
from google.cloud import aiplatform
from datetime import datetime

In [ ]:
# specify parameters
P = ! gcloud config list --format 'value(core.project)'
PROJECT_ID = P[0]
REGION = "us-central1"

SERVICE_ACCOUNT = f"sa-vertex-pipelines@{PROJECT_ID}.iam.gserviceaccount.com"
NETWORK = "vpc-adam-default"

USE_CASE = "custom-training"
ML_FRAMEWORK = "scikit"
MODEL_TYPE = "binclass"
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

# GCS inputs
GCS_BUCKET_NAME = f"bkt-{REGION}-{USE_CASE}"
GCS_BUCKET_PATH = f"gs://{GCS_BUCKET_NAME}"
GCS_BUCKET_PATH_DATA = f"{GCS_BUCKET_PATH}/data"
GCS_BUCKET_PATH_CONFIGS = f"{GCS_BUCKET_PATH}/configs"
GCS_BUCKET_PATH_TMP = f"{GCS_BUCKET_PATH}/tmp"
GCS_BUCKET_PATH_STAGING = f"{GCS_BUCKET_PATH}/staging"

TRAIN_DS = "tab_class_10inps_1krows_tes_3498.csv"

In [ ]:
TRAIN_VERSION  = "scikit-learn-cpu.0-23"
DEPLOY_VERSION = "sklearn-cpu.0-23"

TRAIN_IMAGE = "us-docker.pkg.dev/vertex-ai/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "us-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(DEPLOY_VERSION)

JOB_NAME = "vai_CustomTrainingJob"

CMDARGS = [f"--bucket_name={GCS_BUCKET_NAME}", f"--train_ds={TRAIN_DS}"]

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

job = aiplatform.CustomTrainingJob(
    display_name = JOB_NAME
    , project = PROJECT_ID
    , location = REGION
    , staging_bucket = GCS_BUCKET_PATH
    , script_path = "training_script.py"
    , container_uri = TRAIN_IMAGE
    , requirements = ["gcsfs"]
    , model_serving_container_image_uri = DEPLOY_IMAGE
)

MODEL_DISPLAY_NAME = "model_mymodel"#"vai_model_registry_name"

# Start the training
model = job.run(
    model_display_name = MODEL_DISPLAY_NAME
    , args = CMDARGS
    , replica_count = 1
    , service_account = f"sa-vertex-pipelines@{PROJECT_ID}.iam.gserviceaccount.com"
)

In [ ]:
dir(model)